In [ ]:
# Instalcion de pinecon
!pip install pypdf cohere tiktoken langchain sentence_transformers openai pinecone-client -q

# Miselanea

## Funcion para calcular el costo de los embedding

In [36]:
def print_embeddings_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f'Total Tokens: {total_tokens}')
  print(f'Embeddings consto en dolar:  {total_tokens / 1000 * 0.0001:.5f}')

# Configuraciones iniciales

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()

INDEX_NAME = 'taller'

openai_api_key=os.getenv('OPENAI_API_KEY', 'YourAPIKey')
pinecone_api_key=os.getenv('PINECONE_API_KEY', 'YourAPIKey')
pinecone_environment=os.getenv('PINECONE_ENVIRONMENT', 'YourEnvironment')

# Descarga de archivos necesarios

In [11]:
import requests

# Fichero PDF
archivos_url = ["https://preguntapdf.s3.eu-south-2.amazonaws.com/la-rosaleda_v3.pdf"]

for url in archivos_url:
    doc_to_download = requests.get(url)
    # Guardar fichero
    pdf_file = open(url.split("/")[-1], "wb")
    pdf_file.write(doc_to_download.content)

# Configuracion de pinecone

In [12]:
import pinecone
from langchain.vectorstores import Pinecone

## Creacion del indices

In [20]:
# La dimension y la metric estan dadas por el vector creado por nuesto modelo de embedding -> OpenAI/text-embedding-ada-002
pinecone.create_index(INDEX_NAME, dimension=1536, metric="cosine")
# listar indice
pinecone.list_indexes()

['taller']

In [21]:
# Guardamos su descripcion
collection_description = pinecone.describe_index(INDEX_NAME)

In [22]:
collection_description

IndexDescription(name='taller', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

# Carga de datos

In [23]:
# Paquetes necesarios
from langchain.document_loaders import PyPDFLoader

In [24]:
# Lectura del archivo
FILE = "la-rosaleda_v3.pdf"
loader = PyPDFLoader(FILE)
doc = loader.load()

# Crear chunks

In [25]:
# Paquetes necesarios
from langchain.text_splitter import RecursiveCharacterTextSplitter

## RecursiveCharacterTextSplitter

Anteriormente habiamos visto metodos CharacterTextSplitter para la creacion de chunk o dividir los textos en este caso haremos uso de RecursiveCharacterTextSplitter que no es más que una forma recursiva para dividir los textos, estos nos permite que los fragmentos dividos mantengan un sentido en el contexto general 

In [26]:
# Funcion creadora de chunk
def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )
    return text_splitter.split_documents(doc_to_chunk)

chunks = create_chunks(doc)

# Embedding

In [27]:
# Paquetes necesrios
from langchain.embeddings.openai import OpenAIEmbeddings

## Calcula el costo

In [37]:
print_embeddings_cost(chunks)

Total Tokens: 1621
Embeddings consto en dolar:  0.00016


## Carga el modelo de embedding

In [28]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Carga de los embeddings a pinecone

Pinecone.from_documents es un método en la clase Pinecone de la biblioteca Langchain que se utiliza para inicializar un objeto Pinecone a partir de una lista de documentos y embeddings. Aquí están todos los parámetros que se pueden pasar a este método:

* documents (List[Document]): Una lista de objetos Document que representan los documentos que se agregarán al vectorstore de Pinecone.
* embedding (Embeddings): Un objeto de embeddings que se utilizará para calcular los vectores de los documentos.
* metadatas (Optional[List[dict]]): Una lista opcional de metadatos asociados a los documentos.
* ids (Optional[List[str]]): Una lista opcional de identificadores para los documentos.
* namespace (Optional[str]): Un espacio de nombres opcional para agregar los documentos al vectorstore.
* index_name (Optional[str]): El nombre opcional del índice de Pinecone al que se agregarán los documentos.
* upsert_kwargs (Optional[dict]): Argumentos opcionales adicionales para el método upsert utilizado para agregar los documentos al vectorstore.
* pool_threads (int): El número de subprocesos en el grupo de subprocesos utilizado para agregar los documentos al vectorstore.

In [29]:
Pinecone.from_documents(
    chunks,
    embeddings,
    index_name=INDEX_NAME)

# QaN

load_qa_chain es una función en la biblioteca Langchain que se utiliza para cargar una cadena de procesamiento de preguntas y respuestas (QA). Aquí están los parámetros que se pueden pasar a esta función:

* llm (BaseLanguageModel): El modelo de lenguaje que se utilizará en la cadena de QA.
* chain_type (str): El tipo de cadena de combinación de documentos que se utilizará en la cadena de QA. Puede ser "stuff", "map_reduce", "refine" o "map_rerank".
* verbose (Optional[bool]): Indica si se ejecutarán las cadenas en modo detallado o no.
* **kwargs (Any): Argumentos adicionales que se pueden pasar a la función.

## Tipos de cadenas QaN

* "stuff": Este tipo de cadena de combinación de documentos utiliza un enfoque simple de concatenación de documentos. Los documentos se combinan en un solo texto y se envían al modelo de lenguaje para generar una respuesta.

* "map_reduce": Este tipo de cadena utiliza un enfoque de map-reduce para combinar documentos. Los documentos se dividen en fragmentos más pequeños y se envían al modelo de lenguaje en paralelo. Luego, las respuestas parciales se combinan utilizando una función de reducción para generar una respuesta final.

* "refine": Este tipo de cadena utiliza un enfoque de refinamiento iterativo. Los documentos se combinan inicialmente utilizando el enfoque "stuff" y se genera una respuesta inicial. Luego, se realiza un proceso de refinamiento iterativo donde se seleccionan y agregan documentos adicionales para mejorar la respuesta.

* "map_rerank": Este tipo de cadena utiliza un enfoque de map-reduce con reranking. Los documentos se dividen en fragmentos y se envían al modelo de lenguaje en paralelo. Luego, las respuestas parciales se clasifican y se selecciona la mejor respuesta utilizando un algoritmo de reranking.

## Carga del modelo

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [81]:
# Instacia de la base da dato de vectq
vstore = Pinecone.from_existing_index(INDEX_NAME, embeddings)

# carga de los modelos
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
chainQA = load_qa_chain(llm, chain_type="stuff")

### Pregunta a realizar 

In [107]:
pregunta = "Mi amigo tom es muy fanatico del malago fc sabes que podria regalarle?"

### Consulta en la base de datos

``similarity_search`` es un método en la clase Pinecone de la biblioteca Langchain que se utiliza para buscar documentos similares a una consulta dada en un vectorstore de Pinecone. Aquí están los parámetros que se pueden pasar a este método:

* query (str): El texto de la consulta para buscar documentos similares.
* k (int): El número de documentos similares que se deben devolver. El valor predeterminado es 4.
* filter (Optional[dict]): Un diccionario opcional de argumentos para filtrar los resultados basados en metadatos.
* namespace (Optional[str]): El espacio de nombres opcional en el que se debe realizar la búsqueda. El valor predeterminado es None, lo que significa que se buscará en todos los espacios de nombres.

Es importante tener en cuenta que la similitud se calcula utilizando la métrica de similitud especificada al crear el índice de Pinecone, que puede ser "cosine" u otra métrica compatible.

In [108]:
# Me traera los 3 documentos similares

docs = vstore.similarity_search(pregunta, 3)

In [109]:
docs

[Document(page_content='HISTORIA DEL CLUB\nLos orígenes del fútbol malagueño se remontan a 1904. En 1941, con la inauguración de La Rosaleda, nace el C.D. Málaga. En 1994 coge su testigo el Málaga Club de Fútbol, consiguiendo los mejores resultados deportivos de la historia. Gana la Intertoto en 2002 y disputa la Copa de la UEFA en la 2002/03. Bajo la presidencia de Sheikh Abdullah Al Thani, y tras la histórica cuarta plaza obte-nida en la temporada 2011/12, la entidad de Martiricos disputa la Champions League, alcanzando los cuartos de final.Málaga CFCÓMO LLEGAR AL ESTADIO\nEstadio La Rosaleda,Paseo de Martiricos s/n 29011, Málaga\nParking Centro ComercialRosaleda\nLínea 15 (Paradas 1505 y 1570)\nEstación Málaga María Zambrano (LD 2.8 km) Estación Málaga-Centro Alameda (MD 2.1 km)\nAeropuerto de Málaga', metadata={'page': 0.0, 'source': 'la-rosaleda_v3.pdf'}),
 Document(page_content='Localización ventanilla de incidencias y email de \natención al espectador\nVentanilla de incidencias:

## Cargamos la respues a la cadena QN definida previamente

Pintamos la respues junto con los detalles de la operacion a OpenIa para saber su costo de uso

In [112]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response = chainQA.run(input_documents=docs, question=pregunta)
    print(f"Respuesta ChatGPT: {respuesta} \n {cb}")

Respuesta ChatGPT: El Málaga Club de Fútbol ha ganado una Copa Intertoto en el año 2002. 
 Tokens Used: 776
	Prompt Tokens: 676
	Completion Tokens: 100
Successful Requests: 1
Total Cost (USD): $0.0012140000000000002
Mi amigo tom es muy fanatico del malago fc sabes que podria regalarle? [Document(page_content='HISTORIA DEL CLUB\nLos orígenes del fútbol malagueño se remontan a 1904. En 1941, con la inauguración de La Rosaleda, nace el C.D. Málaga. En 1994 coge su testigo el Málaga Club de Fútbol, consiguiendo los mejores resultados deportivos de la historia. Gana la Intertoto en 2002 y disputa la Copa de la UEFA en la 2002/03. Bajo la presidencia de Sheikh Abdullah Al Thani, y tras la histórica cuarta plaza obte-nida en la temporada 2011/12, la entidad de Martiricos disputa la Champions League, alcanzando los cuartos de final.Málaga CFCÓMO LLEGAR AL ESTADIO\nEstadio La Rosaleda,Paseo de Martiricos s/n 29011, Málaga\nParking Centro ComercialRosaleda\nLínea 15 (Paradas 1505 y 1570)\nEstaci

# QnA con memoria

## Creamos la memoria conversacional

In [133]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Creamos el template del promt
template = """Eres un chatbot manteniendo una conversación con un humano.

Dadas las siguientes partes extraídas de un documento largo y el historial de la conversacion, crea una respuesta final para la pregunta del Human,
Si la respuesta final no se cuentra en el las partes extraídas de un documento largo o en el historial de la conversacion tienes que decir "Lo siento no tengo esa informacion".

## Partes del documento largo
{context}

## Historial de la conversación
{chat_history}

## Pregunta del Human y respuesta final
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

# Creamos la memoria
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

## Cargamos la memoria a la cadena QnA

In [134]:
chainQnAMemory = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=prompt
)

## Realizamos una convercion

In [143]:
# Variable conde se guarda el historial
with get_openai_callback() as cb:
    pregunta = "Crea una porra usando los inventos de nicola tesla"
    docs = vstore.similarity_search(pregunta, 3)
    response = chainQnAMemory({"input_documents": docs, "human_input": pregunta})
    print(cb)

Tokens Used: 1462
	Prompt Tokens: 1447
	Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $0.029240000000000002


In [144]:
print(memory.load_memory_variables({})['chat_history'])

Human: Sabes cual es mi nombre?
AI:  Lo siento, no tengo esa información.
Human: mi nombre es Daniel, necsito darle un regalo a mi amigo tom
AI:  Puedes visitar la tienda de la Rosaleda, ubicada en la fachada exterior (esquina Tribuna con Gol). También puedes visitar la página web del Málaga C.F. para obtener más información sobre el club y sus productos.
Human: Puedes llamar me por mi nombre?, a mi amigo le gusta la ropa donde puedo comprarla?
AI:  Hola Daniel, sí, puedo llamarte por tu nombre. Para comprar ropa para tu amigo, puedes visitar la tienda de la Rosaleda, ubicada en la fachada exterior (esquina Tribuna con Gol). También puedes visitar la página web del Málaga C.F. para obtener más información sobre el club y sus productos.
Human: Crea una porra usando el nombre de mi amigo
AI:  ¡Vamos Tom! ¡Vamos Málaga!
Human: Crea una porra usando los inventos de nicola tesla
AI:  Lo siento, no tengo esa información.
